In [17]:
import numba

In [61]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)
MIN = 10
MAX = 20

def is_ok(pauses, duration, MIN=10, MAX=20):

    pauses = [0, *pauses, duration]

    for start, end in zip(pauses[:-1], pauses[1:]):
        if not MIN <= end - start <= MAX:
            return False
    return True


def splitter(pauses, duration, MIN=10, MAX=20):
    from itertools import combinations, chain

    def powerset(iterable):
        "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    all_combinations = list(powerset(pauses))
    logging.info(f"Testing {len(all_combinations)} combinations....")
    for combination in all_combinations:
        if is_ok(combination, duration, MIN=MIN, MAX=MAX):
            return list(combination)
    logging.warning(f"No solution found so that {MIN=}s <= duration <= {MAX=}s.")
    return None


pauses = [i for i in range(2,40,4)]
splitter(pauses, 40)


2021-12-10 13:42:28,620 - INFO - Testing 1024 combinations....


[10, 22]

In [62]:
splitter([8, 18], 25)

2021-12-10 13:42:30,052 - INFO - Testing 4 combinations....
2021-12-10 13:42:30,053 - WARNING - No solution found so that MIN=10s <= duration <= MAX=20s.


In [ ]:
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_silence
file = "/home/peterr/macocu/task6_speech/data/00009875.flac.wav"
sound_file = AudioSegment.from_wav(file)
audio_chunks = split_on_silence(sound_file, min_silence_len=1000, silence_thresh=-40 )
print([i.duration_seconds for i in audio_chunks])

In [14]:
gold = "Recimo da je to prvi chunk. To drugi. In končno tretji."
model_output = "To je ta drugi."

from fuzzywuzzy import fuzz
def get_subset(gold, model_output):
    results = list()
    gold = gold.split(" ")
    model_output = model_output.split(" ")
    M = len(gold)
    for start in range(M):
        for end in range(M):
            try:
                subset = gold[start:end]
            except IndexError:
                continue
            ratio = fuzz.ratio(" ".join(model_output), " ".join(subset))
            results.append((ratio, start, end, " ".join(subset)))
    maximum_ratio = max(results, key=lambda tup: tup[0])
    for result in results:
        if result[0] == maximum_ratio[0]:
            return result
    return results


get_subset(gold, model_output)


(75, 6, 8, 'To drugi.')

In [66]:
get_subset(
    "a o tome govori i tematika zbog kojeg su se sve žalili",
    "aotomegovoritematika"
)

(87, 0, 6, 'a o tome govori i tematika')

In [57]:
get_subset(
    "poštovane kolegice i kolege zastupnici molio bi da nastavimo sa radom sjednice",
    "poštovane kolegice ikolege zastupnici"
)

(99, 0, 5, 'poštovane kolegice i kolege zastupnici')

In [67]:
import difflib

l1 = "poštovane kolegice i kolege zastupnici molio bi da nastavimo sa radom sjednice"
l2 = "poštovane kolegice ikolege zastupnici"

seq_mat = difflib.SequenceMatcher(a=l1, b=l2)

match = seq_mat.find_longest_match(alo=0, ahi=len(l1), blo=0, bhi=len(l2))

print("============ Longest Matching Sequence ==================")
print("\nMatch Object : {}".format(match))
print("Matching Sequence from l1 : {}".format(l1[match.a:match.a+match.size]))
print("Matching Sequence from l2 : {}\n".format(l2[match.b:match.b+match.size]))

print("============ All Matching Sequences ==================")
for match in seq_mat.get_matching_blocks():
    print("\nMatch Object : {}".format(match))
    print("Matching Sequence from l1 : {}".format(l1[match.a:match.a+match.size]))
    print("Matching Sequence from l2 : {}".format(l2[match.b:match.b+match.size]))

============ Longest Matching Sequence ==================

Match Object : Match(a=0, b=0, size=20)
Matching Sequence from l1 : poštovane kolegice i
Matching Sequence from l2 : poštovane kolegice i

============ All Matching Sequences ==================

Match Object : Match(a=0, b=0, size=20)
Matching Sequence from l1 : poštovane kolegice i
Matching Sequence from l2 : poštovane kolegice i

Match Object : Match(a=21, b=20, size=17)
Matching Sequence from l1 : kolege zastupnici
Matching Sequence from l2 : kolege zastupnici

Match Object : Match(a=78, b=37, size=0)
Matching Sequence from l1 : 
Matching Sequence from l2 : 


In [70]:
!pip install nwalign3

     |████████████████████████████████| 101 kB 3.6 MB/s ta 0:00:011
  Created wheel for nwalign3: filename=nwalign3-0.1.2-cp38-cp38-linux_x86_64.whl size=202828 sha256=c832642efe00eeb061d196410abce5b7c11bc2c97cd62b2e7507727e9bb6cba0
  Stored in directory: /home/peterr/.cache/pip/wheels/3a/9d/62/f90fb1d9ac5c9be659660df5d982343338347d1d70e4252042
Successfully built nwalign3


In [72]:
import nwalign3 as nw

In [78]:
nw.global_align_no_matrix("EEAEE-", "EF-EEG")

TypeError: global_align_no_matrix() takes exactly 5 positional arguments (2 given)

In [1]:
from Bio import pairwise2
from Bio.Seq import Seq
seq1 = Seq("ACCGGT") 
seq2 = Seq("ACGT")

pairwise2.align.globalxx(seq1, seq2)

[Alignment(seqA='ACCGGT', seqB='A-C-GT', score=4.0, start=0, end=6),
 Alignment(seqA='ACCGGT', seqB='AC--GT', score=4.0, start=0, end=6),
 Alignment(seqA='ACCGGT', seqB='A-CG-T', score=4.0, start=0, end=6),
 Alignment(seqA='ACCGGT', seqB='AC-G-T', score=4.0, start=0, end=6)]

In [10]:
alignments = pairwise2.align.globalxx(
        Seq("a o tome govori i tematika zbog kojeg su se sve zalili".encode("ascii")),
        Seq("a o tome govori itematika")
)
alignments

[Alignment(seqA='a o tome govori i tematika zbog kojeg su se sve zalili', seqB='a o tome govori i-temati--------k----------------a----', score=25.0, start=0, end=54),
 Alignment(seqA='a o tome govori i tematika zbog kojeg su se sve zalili', seqB='a o tome govori i-tematik------------------------a----', score=25.0, start=0, end=54)]

In [11]:
alignments[1].seqB

'a o tome govori i-tematik------------------------a----'

In [12]:
l1 = "postovane kolegice i kolege zastupnici molio bi da nastavimo sa radom sjednice"
l2 = "postovane kolegice ikolege zastupnici ja bi molio"

alignments = pairwise2.align.globalxx(
        Seq(l2),
        Seq(l1))
alignments

[Alignment(seqA='postovane kolegice i-kolege zastupnici ja----- bi ----------moli-------o----------', seqB='postovane kolegice i kolege zastupnici --molio bi da nastavimo-- sa radom sjednice', score=45.0, start=0, end=82),
 Alignment(seqA='postovane kolegice i-kolege zastupnici ja---- bi ----------moli-------o----------', seqB='postovane kolegice i kolege zastupnici -molio bi da nastavimo-- sa radom sjednice', score=45.0, start=0, end=81),
 Alignment(seqA='postovane kolegice i-kolege zastupnici ja---- bi ----------moli-------o----------', seqB='postovane kolegice i kolege zastupnici m-olio bi da nastavimo-- sa radom sjednice', score=45.0, start=0, end=81),
 Alignment(seqA='postovane kolegice i-kolege zastupnici j-a--- bi ----------moli-------o----------', seqB='postovane kolegice i kolege zastupnici -molio bi da nastavimo-- sa radom sjednice', score=45.0, start=0, end=81),
 Alignment(seqA='postovane kolegice i-kolege zastupnici ja--- bi ----------moli-------o----------', seqB='postovan

In [13]:
alignments[1].seqB

'postovane kolegice i kolege zastupnici -molio bi da nastavimo-- sa radom sjednice'

In [15]:
get_subset(l1, l2)

(92, 0, 6, 'postovane kolegice i kolege zastupnici molio')

In [16]:
get_subset(l1, "")

(100, 0, 0, '')